In [1]:
from vmtk import vmtkscripts
from extractionmethods import initialization_image, levelset_segmentation, marching_cubes, image_writer, surface_writer, image_reader, image_cast, vessel_enhancer
import numpy as np
import os
import json
from joblib import Parallel, delayed

In [2]:
datasetDir = os.path.join(os.getcwd(), 'multiphoton-sample', 'multiphoton-sample-3')
datasetBaseDir = os.path.join(datasetDir, 'base')
datasetBaseImagePath = os.path.join(datasetBaseDir, 'staxk1.mha')

datasetDerivedDir = os.path.join(datasetDir, 'derived')
if not os.path.exists(datasetDerivedDir):
    os.makedirs(datasetDerivedDir)
    print(f'created derived dataset directory at: {datasetDerivedDir}')

In [3]:
def run_pre_analysis(datasetDerivedDir, lowerThreshValue, iterationValue):
    parameterDir = os.path.join(datasetDerivedDir, f'thresh{lowerThreshValue}'.replace('.', '-'), f'iterations{iterationValue}')
    if not os.path.exists(parameterDir):
        os.makedirs(parameterDir)

    upperThreshValue = None
    curvature = 0
    propagation = 0
    advection = 1
    enhanceMethod = "ved"
    enhanceIterations = 3
    enhanceDiffusionIterations = 3

    image = image_reader(datasetBaseImagePath)
    image_writer(image, os.path.join(parameterDir, 'input-image.vti'))
    
    casted = image_cast(image)
    image_writer(casted, os.path.join(parameterDir, 'casted-image.vti'))
    
    enhanced = vessel_enhancer(casted, enhanceMethod, enhanceIterations, enhanceDiffusionIterations)
    image_writer(enhanced, os.path.join(parameterDir, 'enhanced-image.vti'))

    init = initialization_image(casted, lowerThreshValue, upperThreshValue)
    image_writer(init, os.path.join(parameterDir, 'initialization-image.vti'))
    
    return True

def run_analysis(datasetDerivedDir, lowerThreshValue, iterationValue):
    parameterDir = os.path.join(datasetDerivedDir, f'thresh{lowerThreshValue}'.replace('.', '-'), f'iterations{iterationValue}')
    if not os.path.exists(parameterDir):
        os.makedirs(parameterDir)
        
    upperThreshValue = None
    curvature = 0
    propagation = 0
    advection = 1
    enhanceMethod = "ved"
    enhanceIterations = 3
    enhanceDiffusionIterations = 3
    
    casted = image_reader(os.path.join(parameterDir, 'casted-image.vti'))
    enhanced = image_reader(os.path.join(parameterDir, 'enhanced-image.vti'))
    init = image_reader(os.path.join(parameterDir, 'initialization-image.vti'))

    ls = levelset_segmentation(casted, init, enhanced, curvature, propagation, advection, iterationValue)
    image_writer(ls, os.path.join(parameterDir, 'levelset-image.vti'))

    mc = marching_cubes(ls)
    surface_writer(mc, os.path.join(parameterDir, 'surface.vtp'))

    params = {'initialImagePath': datasetBaseImagePath,
              'parameterDir': parameterDir,
              'lowerThreshValue':lowerThreshValue,
              'upperThreshValue':upperThreshValue,
              'enhanceMethod':enhanceMethod,
              'enhanceIterations': enhanceIterations,
              'enhanceDiffusionIterations':enhanceDiffusionIterations,
              'curvatureScaling':curvature,
              'propagation':propagation,
              'advection':advection,
              'iterationValue':iterationValue,
              'processSteps':['read_image', 'initialize_image', 'cast_image', 'vessel_enhancer', 'level_set_evolution', 'marching_cubes']}
    with open(os.path.join(parameterDir, 'parmas.json'), 'w+') as f:
        json.dump(params, f)

    return True
        

In [4]:
lowerThreshValues = [200, 225, 240, 250, 255]
iterationValues = [50, 100, 200, 350, 500]

In [4]:
Parallel(n_jobs=3)(delayed(run_pre_analysis)(datasetDerivedDir, lowThresh, iteration) for lowThresh in lowerThreshValues for iteration in iterationValues)

Spacing 0.821945 0.821945 2.000000
Spacing 0.821945 0.821945 2.000000
Spacing 0.821945 0.821945 2.000000
Origin 0.000000 0.000000 0.000000
Origin 0.000000 0.000000 0.000000
Origin 0.000000 0.000000 0.000000
Dimensions 512 512 243
Dimensions 512 512 243
Dimensions 512 512 243
Writing VTK XML image file.
Writing VTK XML image file.
Writing VTK XML image file.
image wrote to location /home/rlizzo/projects/blood-vessel-modelling/confocal-multi-vessel-samples/multiphoton-sample/multiphoton-sample-3/derived/thresh200/iterations100/input-image.vti
Writing VTK XML image file.
image wrote to location /home/rlizzo/projects/blood-vessel-modelling/confocal-multi-vessel-samples/multiphoton-sample/multiphoton-sample-3/derived/thresh200/iterations50/input-image.vti
Writing VTK XML image file.
image wrote to location /home/rlizzo/projects/blood-vessel-modelling/confocal-multi-vessel-samples/multiphoton-sample/multiphoton-sample-3/derived/thresh200/iterations200/input-image.vti
Writing VTK XML image fi

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [ ]:
Parallel(n_jobs=12)(delayed(run_analysis)(datasetDerivedDir, lowThresh, iteration) for lowThresh in lowerThreshValues for iteration in iterationValues)

Reading VTK XML image file.
Reading VTK XML image file.
Reading VTK XML image file.
Reading VTK XML image file.
Reading VTK XML image file.
Reading VTK XML image file.
Reading VTK XML image file.
Reading VTK XML image file.
Reading VTK XML image file.
Reading VTK XML image file.
Reading VTK XML image file.
Reading VTK XML image file.
Spacing 0.821945 0.821945 2.000000
Origin 0.000000 0.000000 0.000000
Dimensions 512 512 243
Reading VTK XML image file.
Spacing 0.821945 0.821945 2.000000
Origin 0.000000 0.000000 0.000000
Dimensions 512 512 243
Spacing 0.821945 0.821945 2.000000
Origin 0.000000 0.000000 0.000000
Reading VTK XML image file.
Spacing 0.821945 0.821945 2.000000
Dimensions 512 512 243
Origin 0.000000 0.000000 0.000000
Reading VTK XML image file.
Dimensions 512 512 243
Reading VTK XML image file.
Spacing 0.821945 0.821945 2.000000
Origin 0.000000 0.000000 0.000000
Spacing 0.821945 0.821945 2.000000
Dimensions 512 512 243
Origin 0.000000 0.000000 0.000000
Dimensions 512 512 243
